In [2]:
%cd ..

/home/lcc/projects/MetaHarmonizer


In [3]:
import gevent.monkey
gevent.monkey.patch_all(thread=False, select=False)

True

In [4]:
import numpy as np
import torch
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModel
import os
import pandas as pd
from importlib import reload
import src.models.ontology_mapper_st as om_st
from src.Engine import ontology_mapping_engine as ome
import src.models.ontology_mapper_lm as om_lm
reload(om_st)
reload(ome)
reload(om_lm)

/home/lcc/miniconda3/envs/gsoc/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/lcc/miniconda3/envs/gsoc/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package punkt to /home/lcc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/lcc/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/lcc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/lcc/nltk_data...
[nltk_data]   Package s

<module 'src.models.ontology_mapper_lm' from '/home/lcc/projects/MetaHarmonizer/src/models/ontology_mapper_lm.py'>

In [5]:
cura_map = pd.read_csv('./data/corpus/cbio_treatment_name/trt_name_query_for_NCIT:C1909.csv')
query_list = cura_map['original_value'].tolist()
small_corpus_list = cura_map['curated_ontology'].to_list()
# large_corpus_list = pd.read_csv('./data/corpus_from_NCIT_C1909.csv')

In [6]:
df = pd.read_csv('./data/corpus/cbio_treatment_name/trt_name_query_for_NCIT:C1909.csv')

df["original_value"] = df["original_value"].astype(str).str.strip()
df["curated_ontology"] = df["curated_ontology"].astype(str).str.strip()

cura_map = dict(zip(df["original_value"], df["curated_ontology"]))

In [6]:
other_params = {"test_or_prod": "test"}
onto_engine_large = ome.OntoMapEngine(method='sap-bert', topk=5,
query=query_list, corpus=small_corpus_list,
cura_map=cura_map, 
yaml_path='./src/models/method_model.yaml', 
om_strategy='st', **other_params)

21/05//2025 02:27:1747862830 PM - INFO - OntoMapEngine: Initialized OntoMap Engine module


In [7]:
results_engine_testing = onto_engine_large.run()

21/05//2025 02:27:1747862836 PM - INFO - OntoMapEngine: Running Ontology Mapping
21/05//2025 02:27:1747862836 PM - INFO - OntoMapEngine: Separating exact and non-exact matches
21/05//2025 02:27:1747862836 PM - INFO - OntoMapEngine: Running OntoMap model for non-exact matches
21/05//2025 02:27:1747862836 PM - INFO - OntoMapST: Initialized OntoMap Sentence Transformer module
21/05//2025 02:27:1747862836 PM - INFO - OntoMapST.get_match_results: Creating embeddings for query_list and corpus


No sentence-transformers model found with name cambridgeltl/SapBERT-from-PubmedBERT-fulltext. Creating a new one with MEAN pooling.
No sentence-transformers model found with name cambridgeltl/SapBERT-from-PubmedBERT-fulltext. Creating a new one with MEAN pooling.


21/05//2025 02:27:1747862844 PM - INFO - OntoMapST.get_match_results: Calculating cosine similarity matrix
21/05//2025 02:27:1747862844 PM - INFO - OntoMapST.get_match_results: Generating results table
21/05//2025 02:27:1747862844 PM - INFO - OntoMapST.get_match_results: Results Generated


In [8]:
results_engine_testing

,original_value,curated_ontology,match_level,stage,top1_match,top1_score,top2_match,top2_score,top3_match,top3_score,top4_match,top4_score,top5_match,top5_score
0,Abiraterone,Abiraterone,1,1,Abiraterone,1.0,Abiraterone,1.0,Abiraterone,1.0,Abiraterone,1.0,Abiraterone,1.0
1,Adalimumab,Adalimumab,1,1,Adalimumab,1.0,Adalimumab,1.0,Adalimumab,1.0,Adalimumab,1.0,Adalimumab,1.0
2,Afatinib,Afatinib,1,1,Afatinib,1.0,Afatinib,1.0,Afatinib,1.0,Afatinib,1.0,Afatinib,1.0
3,Alisertib,Alisertib,1,1,Alisertib,1.0,Alisertib,1.0,Alisertib,1.0,Alisertib,1.0,Alisertib,1.0
4,Alpelisib,Alpelisib,1,1,Alpelisib,1.0,Alpelisib,1.0,Alpelisib,1.0,Alpelisib,1.0,Alpelisib,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,vinblastine,Vinblastine,1,2,Vinblastine,1.0000,Vinblastine,1.0000,Vinblastine,1.0000,Vinorelbine,0.7879,Vinorelbine,0.7879
574,vincrisinte,Vincristine,1,2,Vincristine,0.8525,Vincristine,0.8525,Vincristine,0.8525,Vincristine,0.8525,Vincristine,0.8525
575,vincristine,Vincristine,1,2,Vincristine,1.0000,Vincristine,1.0000,Vincristine,1.0000,Vincristine,1.0000,Vincristine,1.0000
576,vincristine sulfate,Vincristine Sulfate,1,2,Vincristine Sulfate,1.0000,Vincristine Sulfate,1.0000,Vincristine,0.8753,Vincristine,0.8753,Vincristine,0.8753


In [10]:
# Sentences Transformer Strategy
total = len(results_engine_testing)

# unmatched（match_level == 99）
unmatched_count = (results_engine_testing['match_level'] == 99).sum()
unmatched_ratio = unmatched_count / total

# matched（match_level != 99）
matched_df = results_engine_testing[results_engine_testing['match_level'] != 99]
matched_count = len(matched_df)
average_match_level = matched_df['match_level'].mean()

print(f"Total records: {total}")
print(f"Unmatched records: {unmatched_count} ({unmatched_ratio:.2%})")
print(f"Matched records: {matched_count} ({average_match_level:.2f})")

Total records: 578
Unmatched records: 114 (19.72%)
Matched records: 464 (1.16)


In [21]:
results_engine_testing.to_csv('results_engine_testing.csv', index=False)

In [ ]:
# st - pubmed-bert
other_params = {"test_or_prod": "test"}
onto_engine_large = ome.OntoMapEngine(method='pubmed-bert', topk=5,
query=query_list, corpus=small_corpus_list,
cura_map=cura_map, 
yaml_path='./src/models/method_model.yaml', 
om_strategy='st', **other_params)

results_engine_testing2 = onto_engine_large.run()

21/05//2025 02:28:1747862916 PM - INFO - OntoMapEngine: Initialized OntoMap Engine module
21/05//2025 02:28:1747862916 PM - INFO - OntoMapEngine: Initialized OntoMap Engine module
21/05//2025 02:28:1747862916 PM - INFO - OntoMapEngine: Running Ontology Mapping
21/05//2025 02:28:1747862916 PM - INFO - OntoMapEngine: Running Ontology Mapping
21/05//2025 02:28:1747862916 PM - INFO - OntoMapEngine: Separating exact and non-exact matches
21/05//2025 02:28:1747862916 PM - INFO - OntoMapEngine: Separating exact and non-exact matches
21/05//2025 02:28:1747862916 PM - INFO - OntoMapEngine: Running OntoMap model for non-exact matches
21/05//2025 02:28:1747862916 PM - INFO - OntoMapEngine: Running OntoMap model for non-exact matches
21/05//2025 02:28:1747862916 PM - INFO - OntoMapST: Initialized OntoMap Sentence Transformer module
21/05//2025 02:28:1747862916 PM - INFO - OntoMapST: Initialized OntoMap Sentence Transformer module
21/05//2025 02:28:1747862916 PM - INFO - OntoMapST.get_match_results

Exception in worker
Traceback (most recent call last):
  File "/home/lcc/miniconda3/envs/gsoc/lib/python3.12/concurrent/futures/thread.py", line 89, in _worker
    work_item = work_queue.get(block=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "src/gevent/queue.py", line 381, in gevent._gevent_cqueue.SimpleQueue.get
  File "src/gevent/queue.py", line 397, in gevent._gevent_cqueue.SimpleQueue.get
  File "src/gevent/queue.py", line 373, in gevent._gevent_cqueue.SimpleQueue._SimpleQueue__get_or_peek
  File "src/gevent/_waiter.py", line 154, in gevent._gevent_c_waiter.Waiter.get
  File "src/gevent/_greenlet_primitives.py", line 65, in gevent._gevent_c_greenlet_primitives.SwitchOutGreenletWithLoop.switch
  File "src/gevent/_gevent_c_greenlet_primitives.pxd", line 35, in gevent._gevent_c_greenlet_primitives._greenlet_switch
gevent.exceptions.LoopExit: This operation would block forever
	Hub: <Hub '' at 0x7f41f4316c00 epoll pending=0 ref=0 fileno=94 thread_ident=0x7f41eeefd640>
	Hand

21/05//2025 02:28:1747862921 PM - INFO - OntoMapST.get_match_results: Calculating cosine similarity matrix
21/05//2025 02:28:1747862921 PM - INFO - OntoMapST.get_match_results: Calculating cosine similarity matrix
21/05//2025 02:28:1747862921 PM - INFO - OntoMapST.get_match_results: Generating results table
21/05//2025 02:28:1747862921 PM - INFO - OntoMapST.get_match_results: Generating results table
21/05//2025 02:28:1747862922 PM - INFO - OntoMapST.get_match_results: Results Generated
21/05//2025 02:28:1747862922 PM - INFO - OntoMapST.get_match_results: Results Generated


In [12]:
# Sentences Transformer Strategy
total = len(results_engine_testing2)

# unmatched（match_level == 99）
unmatched_count = (results_engine_testing2['match_level'] == 99).sum()
unmatched_ratio = unmatched_count / total

# matched（match_level != 99）
matched_df = results_engine_testing2[results_engine_testing2['match_level'] != 99]
matched_count = len(matched_df)
average_match_level = matched_df['match_level'].mean()

print(f"Total records: {total}")
print(f"Unmatched records: {unmatched_count} ({unmatched_ratio:.2%})")
print(f"Matched records: {matched_count} ({average_match_level:.2f})")

Total records: 578
Unmatched records: 168 (29.07%)
Matched records: 410 (1.10)


In [13]:
# lm
other_params = {"test_or_prod": "test"}
onto_engine_large = ome.OntoMapEngine(method='sap-bert', topk=5,
query=query_list, corpus=small_corpus_list,
cura_map=cura_map, 
yaml_path='./src/models/method_model.yaml', 
om_strategy='lm', **other_params)

21/05//2025 02:29:1747862963 PM - INFO - OntoMapEngine: Initialized OntoMap Engine module
21/05//2025 02:29:1747862963 PM - INFO - OntoMapEngine: Initialized OntoMap Engine module
21/05//2025 02:29:1747862963 PM - INFO - OntoMapEngine: Initialized OntoMap Engine module


In [14]:
results_engine_testing3 = onto_engine_large.run()

21/05//2025 02:29:1747862982 PM - INFO - OntoMapEngine: Running Ontology Mapping
21/05//2025 02:29:1747862982 PM - INFO - OntoMapEngine: Running Ontology Mapping
21/05//2025 02:29:1747862982 PM - INFO - OntoMapEngine: Running Ontology Mapping
21/05//2025 02:29:1747862982 PM - INFO - OntoMapEngine: Separating exact and non-exact matches
21/05//2025 02:29:1747862982 PM - INFO - OntoMapEngine: Separating exact and non-exact matches
21/05//2025 02:29:1747862982 PM - INFO - OntoMapEngine: Separating exact and non-exact matches
21/05//2025 02:29:1747862982 PM - INFO - OntoMapEngine: Running OntoMap model for non-exact matches
21/05//2025 02:29:1747862982 PM - INFO - OntoMapEngine: Running OntoMap model for non-exact matches
21/05//2025 02:29:1747862982 PM - INFO - OntoMapEngine: Running OntoMap model for non-exact matches
21/05//2025 02:29:1747862982 PM - INFO - OntoMapLM: Initialized OntoMap Language Model module
21/05//2025 02:29:1747862982 PM - INFO - OntoMapLM.get_match_results: Creating

In [15]:
# Language Model Strategy
total = len(results_engine_testing3)

# unmatched（match_level == 99）
unmatched_count = (results_engine_testing3['match_level'] == 99).sum()
unmatched_ratio = unmatched_count / total

# matched（match_level != 99）
matched_df = results_engine_testing3[results_engine_testing3['match_level'] != 99]
matched_count = len(matched_df)
average_match_level = matched_df['match_level'].mean()

print(f"Total records: {total}")
print(f"Unmatched records: {unmatched_count} ({unmatched_ratio:.2%})")
print(f"Matched records: {matched_count} ({average_match_level:.2f})")

Total records: 578
Unmatched records: 114 (19.72%)
Matched records: 464 (1.16)


In [16]:
print(f"Using om_strategy: {onto_engine_large.om_strategy}")
print(f"Engine class: {type(onto_engine_large._om_model_from_strategy(['test']))}")

Using om_strategy: lm
21/05//2025 02:33:1747863196 PM - INFO - OntoMapLM: Initialized OntoMap Language Model module
21/05//2025 02:33:1747863196 PM - INFO - OntoMapLM: Initialized OntoMap Language Model module
Engine class: <class 'src.models.ontology_mapper_lm.OntoMapLM'>


In [7]:
lm_engine = ome.OntoMapEngine(
    method='sap-bert', topk=5,
    query=query_list, corpus=small_corpus_list,
    cura_map=cura_map, 
    yaml_path='./src/models/method_model.yaml', 
    om_strategy='lm', test_or_prod='test'
)
st_engine = ome.OntoMapEngine(
    method='sap-bert', topk=5,
    query=query_list, corpus=small_corpus_list,
    cura_map=cura_map, 
    yaml_path='./src/models/method_model.yaml', 
    om_strategy='st', test_or_prod='test'
)

lm_model = lm_engine._om_model_from_strategy(['test']).model
st_model = st_engine._om_model_from_strategy(['test']).model

print("LM model type:", type(lm_model))
print("ST model type:", type(st_model))

print("LM model:", lm_model)
print("ST model:", st_model)

22/05//2025 12:47:1747900072 AM - INFO - OntoMapEngine: Initialized OntoMap Engine module
22/05//2025 12:47:1747900073 AM - INFO - OntoMapEngine: Initialized OntoMap Engine module
22/05//2025 12:47:1747900073 AM - INFO - OntoMapEngine: Initialized OntoMap Engine module
22/05//2025 12:47:1747900073 AM - INFO - OntoMapLM: Initialized OntoMap Language Model module
22/05//2025 12:47:1747900076 AM - INFO - OntoMapST: Initialized OntoMap Sentence Transformer module


No sentence-transformers model found with name cambridgeltl/SapBERT-from-PubmedBERT-fulltext. Creating a new one with MEAN pooling.


LM model type: <class 'transformers.models.bert.modeling_bert.BertModel'>
ST model type: <class 'sentence_transformers.SentenceTransformer.SentenceTransformer'>
LM model: BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_fe